# Увидит ли эмбеддинговая модель схожесть между кодом и текстом автотеста???

In [9]:
import pandas as pd
import regex as re
import torch
import math
from torch import Tensor
import torch.nn.functional as F
from typing import List
from transformers import AutoTokenizer, AutoModel

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

test_dataset = pd.read_excel('253951.xlsx')

def prepare_df(df: pd.DataFrame) -> List[pd.DataFrame]:
    def remove_columns_step(dataset: pd.DataFrame) -> pd.DataFrame:
        output_dataset = dataset[
            ["Id", "Direction", "Section", "TestCaseName", "Preconditions", "Steps", "Postconditions",
                 "ExpectedResult"]].copy()
        output_dataset[["Id", "Direction", "Section", "TestCaseName"]] = output_dataset[
                ["Id", "Direction", "Section", "TestCaseName"]].ffill()
        
        return output_dataset

    output_first_step = remove_columns_step(df)

    def parse_tests_by_id(dataset: pd.DataFrame) -> List[pd.DataFrame]:
        test_cases = []
        for group_name, frame in dataset.groupby('Id'):
            test_cases.append(frame)

        return test_cases

    output_second_step = parse_tests_by_id(output_first_step)

    def remove_empty_cells(array: List[pd.DataFrame]) -> List[pd.DataFrame]:
        def up_cells(tst_case: pd.DataFrame, column: str) -> pd.DataFrame:
            for i in range(len(tst_case[column])):
                if i == len(tst_case.index) - 1:
                    tst_case.at[tst_case.index[i], column] = math.nan
                    break
                else:
                    tst_case.at[tst_case.index[i], column] = tst_case.at[tst_case.index[i + 1], column]

            return tst_case

        for dframe in array:
            for col in ["Preconditions", "Steps", "Postconditions", "ExpectedResult"]:
                dframe = up_cells(dframe, col)

            dframe["Steps"] = dframe["Steps"].fillna(dframe["Preconditions"])
            dframe["Steps"] = dframe["Steps"].fillna(dframe["Postconditions"])
            dframe.drop(["Preconditions", "Postconditions"], inplace=True, axis=1)

        return array

    output_third_step = remove_empty_cells(output_second_step)
    return output_third_step

prepared_df = prepare_df(test_dataset)

,Id,Direction,Section,TestCaseName,Steps,ExpectedResult
0,253951.0,РТ_ЕФР/РТ_Тест-кейсы_функциональные/1. Общий ф...,BIQ 12853 (Новый фронт) Личный каб. в ЕФР по з...,Переход в форму с детальной информацией по задаче,Выполнен вход в новую карточку ЕФР,NaN
1,253951.0,РТ_ЕФР/РТ_Тест-кейсы_функциональные/1. Общий ф...,BIQ 12853 (Новый фронт) Личный каб. в ЕФР по з...,Переход в форму с детальной информацией по задаче,Открыт раздел «Личный кабинет» - «Задачи работ...,NaN
2,253951.0,РТ_ЕФР/РТ_Тест-кейсы_функциональные/1. Общий ф...,BIQ 12853 (Новый фронт) Личный каб. в ЕФР по з...,Переход в форму с детальной информацией по задаче,Нажать на название задачи,Открывается блок с детальной информацией по за...
3,253951.0,РТ_ЕФР/РТ_Тест-кейсы_функциональные/1. Общий ф...,BIQ 12853 (Новый фронт) Личный каб. в ЕФР по з...,Переход в форму с детальной информацией по задаче,NaN,NaN


# Векторизация тест-кейса

In [10]:
def process_df(dataset: List[pd.DataFrame] or str) -> List[Tensor] or Tensor:
    def mean_pooling(model_output, attention_mask) -> Tensor:
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    mpnet_vectorized_output_array = []

    model.to(device)
    
    if type(dataset) is list:
        for test_case in dataset:
            # Tokenize
            encoded_input = tokenizer(re.sub(r'\n+', '', str(test_case)), return_tensors='pt', padding=True, truncation=True,
                                      max_length=512).to(device)
            # Compute token embeddings
            with torch.no_grad():
                model_output = model(**encoded_input)

            # Perform pooling. In this case, average pooling
            embedding = mean_pooling(model_output, encoded_input['attention_mask'])
            mpnet_vectorized_output_array.append(F.normalize(embedding, p=2, dim=1))
            
        return mpnet_vectorized_output_array    
    
    elif type(dataset) is str:
        encoded_input = tokenizer(dataset, return_tensors='pt', padding=True, truncation=True,
                                      max_length=512).to(device)
        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Perform pooling. In this case, average pooling
        embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        
        return F.normalize(embedding, p=2, dim=1)
    
    else:
        raise NotImplementedError
    

vectorized_test_case = process_df(prepared_df)
vectorized_test_case[0]

tensor([[-2.6771e-03, -9.5545e-05, -1.1359e-02,  1.8348e-02,  2.0850e-02,
          1.7338e-02,  1.9728e-02,  2.5303e-02,  3.0972e-02,  7.8677e-02,
          6.1934e-02,  3.2747e-02,  3.1847e-02,  1.1097e-01, -1.2086e-02,
         -7.2803e-02,  1.7746e-02, -2.1947e-02, -3.7106e-02,  4.3827e-02,
          5.4526e-02,  1.3394e-03, -5.4130e-03, -2.6022e-02, -9.7015e-03,
         -5.0743e-04, -8.3984e-03,  3.2642e-02,  4.1832e-02,  2.0367e-02,
          4.8569e-02,  3.3681e-02,  4.1542e-03, -9.9981e-02,  1.2349e-02,
         -2.2814e-03, -2.9722e-02, -2.2047e-04, -5.7230e-03, -6.3948e-03,
          3.1178e-02, -8.1521e-02, -1.3933e-02, -1.5467e-02, -5.7738e-02,
         -3.9074e-02, -3.0770e-02,  9.0789e-02,  2.6580e-02,  2.3618e-02,
          4.2688e-03,  5.3476e-02, -1.6033e-02,  4.0568e-02,  8.4588e-02,
         -8.5842e-02, -2.4194e-02,  3.5823e-02, -1.1013e-02, -1.3713e-02,
         -2.6246e-02,  5.6813e-03,  1.8932e-02,  9.5207e-03, -3.2548e-02,
         -7.7591e-03,  9.5704e-02, -8.

In [11]:
type(vectorized_test_case[0])

torch.Tensor

# Векторизация авто-тестов

In [12]:
with open('autotest_with_comments.txt', 'r', encoding='utf-8') as file:
    autotest_with_comments = file.read()

with open('autotest_without_comments.txt', 'r', encoding='utf-8') as file:
    autotest_without_comments = file.read()
    
with open('another_autotest.txt', 'r', encoding='utf-8') as file:
    another_autotest = file.read()
    
type(autotest_with_comments)

package ru.rshb.tests.general_functionality;

import com.codeborne.selenide.Condition;
import io.qameta.allure.Owner;
import lombok.extern.log4j.Log4j2;
import org.openqa.selenium.By;
import org.testng.annotations.Test;
import io.qameta.allure.Description;

import static com.codeborne.selenide.Selenide.$;
import static com.codeborne.selenide.Selenide.open;
import ru.rshb.ai.entities.AIStand;
import ru.rshb.ai.entities.AIWebUser;
import ru.rshb.entities.WebUser;
import ru.rshb.pages.NewEfrLoginPage;
import ru.rshb.ai.pages.PersonalAccountPage;
import main.java.ru.rshb.testitwork.annotations.TestitStatus;

@Log4j2
@Owner("Генеративова Интехия Ионовна")
public class TK253951Test extends GeneralFunctionalityHelper {

    @Test(groups = {"@253951"})
    @Description("TK:253951, АТ:600044")
    @TestitStatus(autoTestId = "600044", autoTestExternalId = "@253951")
    public void Autotest() throws Exception {

        // Русское имя метода в Аллюр
        allureReport.replaceTestMethodName("Пе

str

In [13]:
vectorized_autotest_with_comments = process_df(autotest_with_comments)
vectorized_autotest_without_comments = process_df(autotest_without_comments)
vectorized_another_autotest = process_df(another_autotest)

In [14]:
F.cosine_similarity(vectorized_autotest_without_comments, vectorized_test_case[0])

tensor([0.5281], device='cuda:0')

In [15]:
F.cosine_similarity(vectorized_autotest_with_comments, vectorized_test_case[0])

tensor([0.5364], device='cuda:0')

In [16]:
F.cosine_similarity(vectorized_another_autotest, vectorized_test_case[0])

tensor([0.4819], device='cuda:0')

In [17]:
F.cosine_similarity(vectorized_autotest_with_comments, vectorized_autotest_without_comments)

tensor([0.9929], device='cuda:0')

In [18]:
F.cosine_similarity(vectorized_another_autotest, vectorized_autotest_without_comments)

tensor([0.9702], device='cuda:0')